In [2]:
import os
import pickle
import numpy as np
from PIL import Image
from collections import OrderedDict
import matplotlib
import matplotlib.pyplot as plt

In [3]:
model_path = "C:\\Users\\AndrewHuang\\Documents\\GitHub\\02958343\\02958343\\1a0bc9ab92c915167ae33d942430658c\\model.obj"

In [4]:
vertex_list = []
face_list = []

In [5]:
def get_vertex_face(model_path):
    vertex_list = []
    face_list = []
    with open(model_path, "r") as f:
        for line in f:
            if line.startswith("#") or line.startswith("mtllib") or line.startswith("g") or line.startswith("usemtl"):
                continue
            elif line.startswith("v "):
                #print(line.split())
                x = float(line.split(" ")[1])
                y = float(line.split(" ")[2])
                z = float(line.split(" ")[3])
                vertex_list.append([x, y, z])
                continue
            elif line.startswith("f "):
                v1 = int(line.split(" ")[1].split("/")[0])
                v2 = int(line.split(" ")[2].split("/")[0])
                v3 = int(line.split(" ")[3].split("/")[0])
                face_list.append([v1, v2, v3])
                continue
    return vertex_list, face_list

In [21]:
### done, successfully reconstruct a 3d model without texture
with open("./test_ply.ply", "w") as f:
    f.write("ply\n")
    f.write("format ascii 1.0\n")
    f.write("element vertex " + str(len(vertex_list)) + "\n")
    f.write("property float x\n")
    f.write("property float y\n")
    f.write("property float z\n")
    f.write("element face " + str(len(face_list)) + "\n")
    f.write("property list uint8 uint32 vertex_indices\n")
    f.write("end_header\n")
    for vertex in vertex_list:
        f.write(str(vertex[0]) + " " + str(vertex[1]) + " " + str(vertex[2]) + "\n")
    for face in face_list:
        f.write(str(3) + " " + str(face[0] - 1) + " " + str(face[1] - 1) + " " + str(face[2] - 1) + "\n")

In [108]:
model_list = []
for file in os.listdir("./rotate"):
    if file.startswith("model_name_"):
        print(os.path.join("./rotate", file))
        with open(os.path.join("./rotate", file), "rb") as f:
            model_list_partial = pickle.load(f)
        for model in model_list_partial:
            model_list.append(model.split("/")[-2])
    #break

precision_constant = 10000
    
model_path = "C:\\Users\\AndrewHuang\\Documents\\GitHub\\02958343\\02958343"
for modelidx, model in enumerate(model_list):
    #os.system("blender --background --python 3dloc_2dcoor.py -- --views 18 " + os.path.join(model_path, model, "model.obj"))
    with open("./vertex2pixel", "rb") as f:
        vertex2pixel = pickle.load(f)
    with open("./pixel2vertex", "rb") as f:
        pixel2vertex = pickle.load(f)
    vertex2color = OrderedDict()
    for az in range(18):
        for el in range(3):
            real_img_path = "C:\\tmp\\test\\" + model + "\\" + model + "_az_" + str(az * 20) + "_el_" + str(el * 10) + "_albedo.png0001.png"
            realimg = np.array(Image.open(real_img_path).convert("RGB"))
            realimg_mask = np.array(Image.open(real_img_path))[:, :, 3].reshape([128, 128])
            syn_img_path = "./rotate/rotate/" + str(modelidx * 1000 + az * 10 + el) + ".0.jpg"
            synimg = np.array(Image.open(syn_img_path))
            '''
            for vertex_pixel_pair in vertex2pixel["az_" + str(az * 20) + "_el_" + str(el * 10)]:
                if not repr((precision_constant * np.array(vertex_pixel_pair[:3])).astype(int)) in vertex2color:
                    vertex2color[repr((precision_constant * np.array(vertex_pixel_pair[:3])).astype(int))] = []
                vertex2color[repr((precision_constant * np.array(vertex_pixel_pair[:3])).astype(int))].append(realimg[vertex_pixel_pair[3], vertex_pixel_pair[3]])
            '''
            for i in range(128):
                for j in range(128):
                    if pixel2vertex[az, el, i, j, 0] != 1234567.0:
                        if realimg_mask[i, j] < 255 / 2:
                            print("pass!")
                            continue
                        if (synimg[i, j] == 0).any():
                            continue
                        if not repr((10000 * pixel2vertex[az, el, i, j, 1:]).astype(int)) in vertex2color:
                            vertex2color[repr((precision_constant * pixel2vertex[az, el, i, j, 1:]).astype(int))] = []
                        #vertex2color[repr((precision_constant * pixel2vertex[az, el, i, j, 1:]).astype(int))].append(realimg[i, j])
                        vertex2color[repr((precision_constant * pixel2vertex[az, el, i, j, 1:]).astype(int))].append(synimg[i, j])
            
    vertex_list, face_list = get_vertex_face(os.path.join(model_path, model, "model.obj"))
    color_list = np.zeros(shape=(len(vertex_list), 3)).astype(int)
    for vertexidx, vertex in enumerate(vertex_list):
        if repr((np.array(vertex) * precision_constant).astype(int)) in vertex2color:
            candidate_color_list = vertex2color[repr((np.array(vertex) * precision_constant).astype(int))]
            mean = np.mean(candidate_color_list, axis=0).astype(int)
            std = np.std(candidate_color_list, axis=0).astype(int)
            for coloridx, candidate_color in enumerate(candidate_color_list):
                if (candidate_color > mean + 1 * std).all() or (candidate_color < mean - 1 * std).all():
                    candidate_color_list[coloridx] = mean
            color = np.mean(candidate_color_list, axis=0).astype(int)
            color_list[vertexidx] = color
    '''
    vertex2neighbor = np.zeros(shape=(len(vertex_list), 2)) # idx, dist
    vertex2neighbor[:, 1] = 2.0
    for face in face_list:
        v1, v2, v3 = face[0] - 1, face[1] - 1, face[2] - 1;
        d12 = np.linalg.norm(np.array(vertex_list[v1]) - np.array(vertex_list[v2]))
        d23 = np.linalg.norm(np.array(vertex_list[v2]) - np.array(vertex_list[v3]))
        d31 = np.linalg.norm(np.array(vertex_list[v3]) - np.array(vertex_list[v1]))
        if d12 < vertex2neighbor[v1, 1]:
            vertex2neighbor[v1, 0] = v2
            vertex2neighbor[v1, 1] = d12
        if d12 < vertex2neighbor[v2, 1]:
            vertex2neighbor[v2, 0] = v1
            vertex2neighbor[v2, 1] = d12
        if d23 < vertex2neighbor[v2, 1]:
            vertex2neighbor[v2, 0] = v3
            vertex2neighbor[v2, 1] = d23
        if d23 < vertex2neighbor[v3, 1]:
            vertex2neighbor[v3, 0] = v2
            vertex2neighbor[v3, 1] = d23
        if d31 < vertex2neighbor[v1, 1]:
            vertex2neighbor[v1, 0] = v3
            vertex2neighbor[v1, 1] = d31
        if d31 < vertex2neighbor[v3, 1]:
            vertex2neighbor[v3, 0] = v1
            vertex2neighbor[v3, 1] = d31
    '''
    vertex2neighbor = OrderedDict()
    for face in face_list:
        v1, v2, v3 = face[0] - 1, face[1] - 1, face[2] - 1;
        if str(v1) not in vertex2neighbor:
            vertex2neighbor[str(v1)] = set()
        if str(v2) not in vertex2neighbor:
            vertex2neighbor[str(v2)] = set()
        if str(v3) not in vertex2neighbor:
            vertex2neighbor[str(v3)] = set()
        vertex2neighbor[str(v1)].add(v2)
        vertex2neighbor[str(v1)].add(v3)
        vertex2neighbor[str(v2)].update([v1, v3])
        vertex2neighbor[str(v3)].update([v1, v2])
    cnt_loop = 0
    while True:
        cnt_loop += 1
        all_vertex_not_colored = True
        for vertexidx, vertex in enumerate(vertex_list):
            if np.sum(color_list[vertexidx]) < 10:
                color = np.array([0, 0, 0])
                cnt = 0
                for vertex in vertex2neighbor[str(vertexidx)]:
                    if np.sum(color_list[vertex]) >= 10:
                        color += color_list[vertex]
                        cnt += 1
                if cnt != 0:
                    color_list[vertexidx] = np.array(color) / cnt
                all_vertex_not_colored = False
        if all_vertex_not_colored:
            break
        if cnt_loop >= 100:
            break
            

    with open("./test_ply_syn_mindist_propagation_abnormal_mask.ply", "w") as f:
        f.write("ply\n")
        f.write("format ascii 1.0\n")
        f.write("element vertex " + str(len(vertex_list)) + "\n")
        f.write("property float x\n")
        f.write("property float y\n")
        f.write("property float z\n")
        f.write("property uchar red\n")
        f.write("property uchar green\n")
        f.write("property uchar blue\n")
        f.write("element face " + str(len(face_list)) + "\n")
        f.write("property list uint8 uint32 vertex_indices\n")
        f.write("end_header\n")
        for vertexidx, vertex in enumerate(vertex_list):
            color = color_list[vertexidx]
            f.write(str(vertex[0]) + " " + str(vertex[1]) + " " + str(vertex[2]) + " " + str(color[0]) + " " + str(color[1]) + " " + str(color[2]) + "\n")
        for face in face_list:
            f.write(str(3) + " " + str(face[0] - 1) + " " + str(face[1] - 1) + " " + str(face[2] - 1) + "\n")
    if modelidx >= 0:
        break

./rotate\model_name_0
./rotate\model_name_1000
./rotate\model_name_2000
./rotate\model_name_3000
./rotate\model_name_5000
./rotate\model_name_6000
./rotate\model_name_7000
./rotate\model_name_8000
./rotate\model_name_9000
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass

pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!


In [95]:
cnt = 0
for color in color_list:
    if np.sum(color) == 0.0:
        cnt += 1
print(cnt)

0


In [96]:
if True:
    for idx, vertex in enumerate(vertex_list):
        if repr((np.array(vertex) * precision_constant).astype(int)) in vertex2color:
            candidate_color_list = vertex2color[repr((np.array(vertex) * precision_constant).astype(int))]
            mean = np.mean(candidate_color_list, axis=0)
            std = np.std(candidate_color_list, axis=0)
            print(mean, std)

[194.44897959 102.87755102 106.26530612] [9.74288365 4.32460489 4.29687565]
[188.16666667 116.08333333 112.11111111] [ 8.35497191 15.02890733 13.1256981 ]
[189.8125 125.75   118.3125] [ 4.22557319 19.91701534 19.85995893]
[193.27272727 106.72727273 107.27272727] [ 8.13623667 16.2373918  16.64108229]
[183.625   121.59375 114.75   ] [21.13017688 43.30770852 37.18450618]
[192.33333333 110.63194444 109.11111111] [12.39847661 27.18054047 28.28765381]
[185.640625 113.375    111.0625  ] [12.80250182 23.62698404 18.30033726]
[188.55029586 103.12426036 101.49112426] [ 8.8498241   9.27325226 10.06096747]
[215.5 101.5 102. ] [10.5  1.5  1. ]
[194.72222222 120.27777778 116.75      ] [12.10359094 18.96919205 14.38677054]
[189.546875  117.796875  116.0390625] [16.57728556 21.11727276 20.25042046]
[195.  117.6 112.8] [ 6.41872261 24.28662183 19.85346317]
[191.32544379 113.30177515 110.24852071] [20.85409255 20.82089698 21.43207355]
[188.66666667 104.83333333 102.16666667] [11.13053857 17.77091882 13.

[ 9. 22. 15.] [0. 0. 0.]
[167.58333333  95.58333333  93.13888889] [46.37924883 10.62393786 10.63729318]
[193.16153268 103.62509391 102.32381668] [10.24344159  5.29765987  3.82620663]
[202.         104.         103.83333333] [6.5064071  3.87298335 4.33653728]
[188.42361111 118.04861111 113.89583333] [16.3800868  30.71641606 28.84936783]
[180.28 114.16 111.24] [12.84591764  7.2579887   8.28531231]
[202.171875 126.9375   126.09375 ] [12.38394366 24.19476635 21.1570347 ]
[6. 4. 5.] [0. 0. 0.]
[132. 134. 128.] [108. 108. 103.]
[207. 149. 137.] [0. 0. 0.]
[39.1875 36.1875 35.3125] [30.80349767 28.19983101 26.7986561 ]
[191.55555556 161.44444444 157.77777778] [41.22237197 60.67358528 55.35430214]
[157.91836735 107.24489796 104.73469388] [40.89636031 18.58566398 17.4304831 ]
[200.66666667 152.66666667 151.77777778] [41.90730512 71.92743257 68.06115623]
[154.77777778 126.88888889 124.05555556] [55.30434872 38.57924344 39.14901099]
[213.04 184.24 180.4 ] [23.77406991 49.97989996 52.29378548]
[19

[185.45679012 157.98765432 161.16049383] [28.67821781 50.57478794 48.23499606]
[242. 175. 192.] [0. 0. 0.]
[216.88888889 216.88888889 214.33333333] [9.97899028 6.03896811 4.10960934]
[191.49382716 171.39506173 168.92592593] [39.18994969 48.86629981 48.6907038 ]
[178.12244898 147.97959184 153.53061224] [25.99758102 35.05900304 33.15839839]
[161.34375 160.3125  157.375  ] [28.46288359 37.37034879 36.27649894]
[227.30612245 211.24489796 211.31632653] [22.26351964 34.06233261 29.55353785]
[222.34375 224.84375 223.625  ] [15.76224103  9.82239748 10.34332514]
[226.61333333 220.96888889 220.23111111] [17.0666875  17.3139527  17.99952043]
[177.62049861 170.07202216 171.32686981] [28.75362806 28.22399733 27.41407113]
[135.07266436 130.40138408 128.87197232] [40.55515692 37.71602267 36.92116596]
[153.77 141.99 144.83] [24.30164809 15.26456354 17.10362827]
[227.29333333 215.30666667 212.52      ] [15.31265853 22.42861664 20.26264872]
[226.29752066 212.00826446 210.57024793] [17.3396308  13.263862

[197. 105. 106.] [0. 0. 0.]
[200. 104. 108.] [0. 0. 0.]
[204.33333333 115.33333333 115.        ] [ 6.12825877  9.87702159 10.19803903]
[205.77777778 113.77777778 115.77777778] [2.2825154  5.19496432 1.72848324]
[192. 103.  97.] [0. 0. 0.]
[196.  94.  89.] [0. 0. 0.]
[197. 106. 101.] [0. 0. 0.]
[198. 100.  93.] [5. 8. 8.]
[202. 105.  99.] [0. 0. 0.]
[205. 106. 100.] [0. 0. 0.]
[206. 101.  95.] [0. 0. 0.]
[189.  102.5 100.5] [0.  3.5 1.5]
[179.  99. 100.] [0. 0. 0.]
[191.33333333 115.         112.66666667] [10.20892855  8.83176087 10.20892855]
[194. 104. 106.] [0. 0. 0.]
[188.75 119.25 120.  ] [7.94905655 8.58414236 7.96868873]
[187.24 104.96 103.68] [15.41396769  8.12935422 11.67602672]
[210. 105.  99.] [0. 0. 0.]
[189.71428571 110.04081633 104.67346939] [11.5475945   5.68600875  4.07846816]
[180. 131. 134.] [0. 0. 0.]
[198.  99. 102.] [0. 0. 0.]
[203.  101.5  99.5] [7.  3.5 2.5]
[195.66666667 103.66666667 101.66666667] [2.86744176 3.39934634 0.47140452]
[255. 251. 236.] [0. 0. 0.]
[172

[195.72222222 134.         133.5       ] [10.10714208 46.6190233  41.28962743]
[181.4   99.92  96.76] [16.90680336  9.34374657 10.27961089]
[165.26530612  94.04081633  91.97959184] [41.82194861 19.65752169 18.31031963]
[200.4 107.  106.2] [11.25344392  7.40270221  6.7941151 ]
[170.36734694  92.3877551   92.65306122] [40.74673659 15.40468943 13.334201  ]
[220. 143. 143.] [21. 37. 42.]
[188.5   100.25   94.875] [10.64189833  4.54835135  4.77460731]
[185.         114.5        111.83333333] [21.74089848 19.68713624 20.83599983]
[199.63888889 114.19444444 109.27777778] [ 4.7885054  13.21069175 11.22730908]
[198.95061728 112.71604938 108.62962963] [ 7.30937184 14.60265785 10.7765686 ]
[171.33333333  99.          96.66666667] [50.4667107   5.09901951  7.58653778]
[190.66666667 110.66666667 103.66666667] [ 9.84321537 25.95294888 27.62044331]
[194.25 117.   116.5 ] [ 3.34477204 12.38951169  7.76208735]
[190.         134.33333333 131.        ] [ 4.24264069 36.35320557 31.37939876]
[184.72 125.96

...

[173.8  125.76 131.6 ] [39.14281543 47.15173804 43.90717481]
[103.  119.5 110. ] [34.  34.5 38. ]
[ 7. 13.  9.] [0. 0. 0.]
[184. 105. 108.] [0. 0. 0.]
[194.5 141.  143.5] [ 7.5 41.  37.5]
[192. 118. 117.] [0. 0. 0.]
[243. 255. 254.] [0. 0. 0.]
[202. 131. 127.] [0. 0. 0.]
[190. 112. 100.] [0. 0. 0.]
[189.88888889 111.55555556 109.55555556] [13.29253016 11.64071187 10.96233732]
[34.5 12.5  9. ] [1.5 2.5 5. ]
[6. 6. 6.] [0. 0. 0.]
[5. 5. 5.] [0. 0. 0.]
[ 7.5 10.   6.5] [3.5 6.  2.5]
[154.  89.  85.] [0. 0. 0.]
[184.16666667 132.         133.16666667] [18.28858174 23.48758537 20.50541934]
[195.57142857 154.53061224 151.95918367] [35.38303525 48.42214239 48.18844897]
[ 1. 10.  5.] [0. 0. 0.]
[153. 157. 156.] [0. 0. 0.]
[144. 142. 145.] [0. 0. 0.]
[183. 173. 172.] [0. 0. 0.]
[189.66666667 140.44444444 137.55555556] [ 1.69967317 13.89466547 19.03083917]
[189.66666667 140.44444444 137.55555556] [ 1.69967317 13.89466547 19.03083917]
[144.375  92.375  90.5  ] [29.35797379 15.00468677 15.49596722

[184.04132231 112.66942149 108.62809917] [12.85136457  6.69786657  7.10217949]
[197.55555556 115.08333333 112.80555556] [ 8.92181402 14.31466808 14.30647246]
[234.4425 225.35   222.7125] [12.07185026 14.96754823 16.37304855]
[196.69230769 101.84615385 101.84615385] [6.92136731 4.32961455 3.41593897]
[197.22222222 121.22222222 118.11111111] [11.3278854  20.17944193 20.02282648]
[195.625 105.125 102.875] [7.12280668 4.70206072 3.7893766 ]
[193.52941176 105.47058824 103.76470588] [12.98468622  6.85237311  9.16854862]
[206.97530864 133.64197531 132.86419753] [16.28996323 46.41427273 48.74143807]
[228.125 172.875 168.125] [11.49116508 32.86026286 27.4462543 ]
[198.06122449 116.36734694 111.20408163] [13.02897805 18.06029053 18.13391345]
[221.12 172.36 169.2 ] [15.55820041 21.25743164 18.0930926 ]
[195.59279778 110.3601108  112.46537396] [10.03046594 20.86959394 26.38939989]
[190.94897959 106.8622449  108.42346939] [13.16741352  7.8715297  11.13436569]
[18.  7.  3.] [0. 0. 0.]
[185. 111. 112

[198.2752 113.2816 111.9264] [ 8.04233853 16.84186205 15.84852167]
[202.20415225 106.42560554 103.44982699] [ 7.22855673 11.93771626  9.0083739 ]
[187.09090909 105.81818182 103.63636364] [14.33022581 11.79227931 11.52288417]
[195.     106.     105.5625] [ 9.0346555   9.9435909  12.33373803]
[194.34090909 110.58884298 109.45041322] [ 6.20737531 16.13733833 16.55157178]
[192.66145833 105.25694444 104.90798611] [14.34836659  8.17969188  9.2650183 ]
[193.61284722 108.68576389 108.94618056] [12.45963668  9.85310203 10.79660225]
[192.78715815 105.81807372 104.84423306] [12.2690233   7.16978307  8.37937944]
[192.66942149 108.97107438 109.00826446] [11.24023227 11.20028497 13.05205175]
[190.90022676 105.60997732 105.55328798] [12.53076038  8.00278835  8.38122907]
[192.7216 109.848  109.2176] [15.20363527 15.10436017 16.12592082]
[192.47916667 116.68229167 114.11458333] [13.74051125 18.73068377 17.22728665]
[195.01133787 110.8276644  109.41496599] [13.8993086  18.86110769 18.40627269]
[191.4814

[244. 232. 242.] [0. 0. 0.]
[202.  98.  99.] [0. 0. 0.]
[230.7244898  213.03571429 212.13265306] [22.88196766 46.9450531  45.55743065]
[234.75510204 224.45918367 223.7244898 ] [15.20196191 38.91303485 38.00565649]
[243.828125 241.140625 240.328125] [8.53964502 9.04922249 7.86431256]
[226.40740741 192.88580247 190.83024691] [24.64878119 65.93201302 65.43922633]
[235.33333333 227.91666667 230.33333333] [17.49920633 27.26477683 24.79695322]
[224.7654321  204.38271605 202.19753086] [22.643261   44.88229114 43.56307499]
[230.20710059 213.01183432 212.26035503] [22.72591804 52.61641272 52.11897333]
[244.33333333 245.75       239.44444444] [7.29535621 9.06343386 7.6368179 ]
[245. 245. 247.] [0. 0. 0.]
[217.  171.  167.5] [ 9.  76.  70.5]
[206.44444444 144.77777778 146.33333333] [ 8.49545993 31.94246371 34.19876541]
[244.75 241.5  240.75] [3.69966215 1.5        3.34477204]
[237.4 243.  240.2] [6.08604962 5.83095189 4.44522215]
[242.52 239.72 240.64] [6.50243031 2.89717103 3.28      ]
[231.76 2

[195.65217391 119.03969754 113.35160681] [12.12474543 22.85295534 22.20282378]
[183. 104. 110.] [0. 0. 0.]
[189.         112.         111.66666667] [12.08304597  6.4807407   5.79271573]
[193.02469136 115.09259259 109.17901235] [10.60770455 19.05217189 19.71181742]
[197.49 117.83 113.02] [18.95293909 21.69140613 18.61761531]
[191.03555556 116.66666667 110.22666667] [14.06966512 17.19560667 16.66211138]
[194.0625 124.     118.    ] [ 8.98240468 25.66856054 27.38612788]
[193.81818182 127.9338843  123.7768595 ] [ 8.34285957 27.14261223 26.38968117]
[151.86419753 100.4691358   97.35802469] [36.84813969 29.62716554 30.60818264]
[192.66666667 118.         117.55555556] [32.39684484 27.06781607 28.01102076]
[193.4  131.24 127.64] [23.9382539  45.47164391 47.21226959]
[169.5         93.05555556  89.05555556] [28.28279807 14.01509327 13.2111298 ]
[186.5 104.5  97.5] [4.5 1.5 1.5]
[115.5 115.  114. ] [16.5 17.  18. ]
[130. 130. 130.] [0. 0. 0.]
[166.         138.33333333 139.66666667] [25.9358182

In [98]:
vertex2color

OrderedDict([('array([ 583, 1362, 1241])',
              [array([92, 92, 92], dtype=uint8),
               array([209, 198, 206], dtype=uint8),
               array([147.25, 132.5 , 131.5 ]),
               array([138, 133, 129], dtype=uint8),
               array([109,  55,  43], dtype=uint8),
               array([147.25, 132.5 , 131.5 ]),
               array([126, 103, 111], dtype=uint8),
               array([219, 208, 202], dtype=uint8)]),
             ('array([ 546, 1391, 1241])',
              [array([114, 114, 114], dtype=uint8),
               array([133.23076923, 119.69230769, 118.46153846]),
               array([133.23076923, 119.69230769, 118.46153846]),
               array([69, 69, 69], dtype=uint8),
               array([83, 80, 75], dtype=uint8),
               array([153,  95,  83], dtype=uint8),
               array([68, 64, 65], dtype=uint8),
               array([139, 120, 122], dtype=uint8),
               array([162, 146, 146], dtype=uint8),
               array

In [ ]:
# test_ply_real: threshold = 1.0, with min dist, real pixel, const=10000
# test_plt_syn: threshold = 1.0, with min dist, syn pixel, const=10000
# with min dist:threshold = 1.0 or 0.2 -> blurry, but why? 
#    - instable ray_cast?
# threshold = 0.2: more vertices left blank, with const=1e6
#    - fixed with const=10000, see test_ply_real_threshold20
# 0615: epoch=3, lossGB about 2.0, run with real depth image

In [104]:
### test mask
for modelidx, model in enumerate(model_list):
    #os.system("blender --background --python 3dloc_2dcoor.py -- --views 18 " + os.path.join(model_path, model, "model.obj"))
    with open("./vertex2pixel", "rb") as f:
        vertex2pixel = pickle.load(f)
    with open("./pixel2vertex", "rb") as f:
        pixel2vertex = pickle.load(f)
    vertex2color = OrderedDict()
    for az in range(18):
        for el in range(3):
            real_img_path = "C:\\tmp\\test\\" + model + "\\" + model + "_az_" + str(az * 20) + "_el_" + str(el * 10) + "_albedo.png0001.png"
            realimg = np.array(Image.open(real_img_path).convert("RGB"))
            realimg_mask = np.array(Image.open(real_img_path))[:, :, 3].reshape([128, 128])
            syn_img_path = "./rotate/rotate/" + str(modelidx * 1000 + az * 10 + el) + ".0.jpg"
            synimg = np.array(Image.open(syn_img_path))
            '''
            for vertex_pixel_pair in vertex2pixel["az_" + str(az * 20) + "_el_" + str(el * 10)]:
                if not repr((precision_constant * np.array(vertex_pixel_pair[:3])).astype(int)) in vertex2color:
                    vertex2color[repr((precision_constant * np.array(vertex_pixel_pair[:3])).astype(int))] = []
                vertex2color[repr((precision_constant * np.array(vertex_pixel_pair[:3])).astype(int))].append(realimg[vertex_pixel_pair[3], vertex_pixel_pair[3]])
            '''
            for i in range(128):
                for j in range(128):
                    if pixel2vertex[az, el, i, j, 0] != 1234567.0:
                        if realimg_mask[i, j] < 255 / 2:
                            print("pass!")
                            continue
                        if (synimg[i, j] == 0).any():
                            continue
                        if not repr((10000 * pixel2vertex[az, el, i, j, 1:]).astype(int)) in vertex2color:
                            vertex2color[repr((precision_constant * pixel2vertex[az, el, i, j, 1:]).astype(int))] = []
                        #vertex2color[repr((precision_constant * pixel2vertex[az, el, i, j, 1:]).astype(int))].append(realimg[i, j])
                        vertex2color[repr((precision_constant * pixel2vertex[az, el, i, j, 1:]).astype(int))].append(synimg[i, j])
    vertex_list, face_list = get_vertex_face(os.path.join(model_path, model, "model.obj"))
    color_list = np.zeros(shape=(len(vertex_list), 3)).astype(int)
    for vertexidx, vertex in enumerate(vertex_list):
        if repr((np.array(vertex) * precision_constant).astype(int)) in vertex2color:
            candidate_color_list = vertex2color[repr((np.array(vertex) * precision_constant).astype(int))]
            mean = np.mean(candidate_color_list, axis=0).astype(int)
            std = np.std(candidate_color_list, axis=0).astype(int)
            for coloridx, candidate_color in enumerate(candidate_color_list):
                if (candidate_color > mean + 1 * std).all() or (candidate_color < mean - 1 * std).all():
                    candidate_color_list[coloridx] = mean
            color = np.mean(candidate_color_list, axis=0).astype(int)
            color_list[vertexidx] = color
    if modelidx >= 0:
        break

pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass

pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!


In [107]:
cnt = 0
for color in color_list:
    if np.sum(color) <= 10.0:
        cnt += 1
print(cnt)

3157


In [94]:
realimg_mask[64]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  26, 161,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       144,   7,   0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=uint8)